In [ ]:
%pip install pyhive

In [ ]:
from pyhive import hive


# Connect to Hive Metastore
conn = hive.Connection(host='10.11.0.10', port=10000, username='anonymous')

cursor = conn.cursor()
cursor.execute('SHOW DATABASES')

print("Databases:")
for database in cursor.fetchall():
    print(database)

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("GCS_to_Minio_Transform")
         # Enable Hive support so we can integrate with Hive metastore:
         .enableHiveSupport()
         # GCS configuration (assuming you've put the GCS connector jar in the classpath)
         .config("spark.hadoop.google.cloud.auth.service.account.enable", "true")
         .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/developer/keys/gcp-keys/advocado-billing/adhoc-billing-advocado-billing--5e634048a179.json")
         # Minio configuration
         .config("spark.hadoop.fs.s3a.endpoint", "https://minio.local-stl.n90.co:9000")
         .config("spark.hadoop.fs.s3a.access.key", "5flC58yYrT3qHV8OA0l7")
         .config("spark.hadoop.fs.s3a.secret.key", "Y0tDcY4evyUvSqX6NlAoWsD4Rkn2VIdb4uH2lGu2")
         .config("spark.hadoop.fs.s3a.path.style.access", "true")
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .getOrCreate())

In [ ]:
df = spark.read.parquet("gs://my-gcs-bucket/path/to/data/")